<a href="https://colab.research.google.com/github/Coolinglass/Applied-Machine-Learning-Projects/blob/master/10_6_Next_word_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps to build the next word recommender system

1. Loading and exploring the dataset
2. Creating N-grams of the dialogue
3. Building the N-gram Language Model
4. Predicting the next word using N-gram Language Model

## 1. Loading and exploring the dataset

In [ ]:
# loading the required libraries
import pandas as pd
import numpy as np
import re
import pickle
import random
from tqdm import tqdm

In [ ]:
# open text file and read in data
dialogs = pd.read_csv("sample_reuters_dataset.csv")

In [ ]:
# number of text sequences
len(dialogs)

10000

In [ ]:
# print 10 random dialogs
dialogs.head()


,sentence_number,sentence_text
0,0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...
1,1,They told Reuter correspondents in Asian capit...
2,2,But some exporters said that while the conflic...
3,3,The U . S . Has said it will impose 300 mln dl...
4,4,Unofficial Japanese estimates put the impact o...


In [ ]:
dialogs.drop('sentence_number', axis = 1, inplace = True)

In [ ]:
dialogs[0]

KeyError: 0

In [ ]:
# text cleaning
dialogs_clean = []

for i in dialogs['sentence_text']:
  # remove everything except alphabets, ' and white spaces
  i = re.sub("[^a-zA-Z0-9' ]", "", i)
  # convert text to lowercase
  i = i.lower()
  # add cleaned text to the list
  dialogs_clean.append(i)

In [ ]:
dialogs.head()

,sentence_text
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...
1,They told Reuter correspondents in Asian capit...
2,But some exporters said that while the conflic...
3,The U . S . Has said it will impose 300 mln dl...
4,Unofficial Japanese estimates put the impact o...


In [ ]:
dialogs_clean

["asian exporters fear damage from u  s  japan rift mounting trade friction between the u  s  and japan has raised fears among many of asia ' s exporting nations that the row could inflict far  reaching economic damage  businessmen and officials said ",
 'they told reuter correspondents in asian capitals a u  s  move against japan might boost protectionist sentiment in the u  s  and lead to curbs on american imports of their products ',
 "but some exporters said that while the conflict would hurt them in the long  run  in the short  term tokyo ' s loss might be their gain ",
 "the u  s  has said it will impose 300 mln dlrs of tariffs on imports of japanese electronics goods on april 17  in retaliation for japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost ",
 'unofficial japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports of products hit b

In [ ]:
# creating the vocabulary
# get list of all the words
all_words = " ".join(dialogs_clean).split()

words_dict = {}

# add word-count pair to the dictionary
for word in all_words:
    # check if the word is already in dictionary
    if word in words_dict:
        # increment count of word by 1
        words_dict[word] = words_dict[word] + 1
    else:
        # add the word to dictionary with count 1
        words_dict[word] = 1

In [ ]:
# word dictionary
words_dict

{'asian': 13,
 'exporters': 52,
 'fear': 8,
 'damage': 29,
 'from': 1369,
 'u': 1117,
 's': 2847,
 'japan': 441,
 'rift': 1,
 'mounting': 5,
 'trade': 549,
 'friction': 8,
 'between': 191,
 'the': 12496,
 'and': 4599,
 'has': 974,
 'raised': 70,
 'fears': 13,
 'among': 44,
 'many': 54,
 'of': 6671,
 'asia': 14,
 "'": 2094,
 'exporting': 12,
 'nations': 71,
 'that': 1376,
 'row': 3,
 'could': 291,
 'inflict': 1,
 'far': 55,
 'reaching': 7,
 'economic': 244,
 'businessmen': 15,
 'officials': 190,
 'said': 4649,
 'they': 518,
 'told': 237,
 'reuter': 27,
 'correspondents': 3,
 'in': 5070,
 'capitals': 3,
 'a': 4410,
 'move': 101,
 'against': 270,
 'might': 59,
 'boost': 45,
 'protectionist': 22,
 'sentiment': 10,
 'lead': 96,
 'to': 6337,
 'curbs': 12,
 'on': 1643,
 'american': 126,
 'imports': 242,
 'their': 230,
 'products': 200,
 'but': 650,
 'some': 278,
 'while': 164,
 'conflict': 3,
 'would': 926,
 'hurt': 11,
 'them': 58,
 'long': 119,
 'run': 21,
 'short': 87,
 'term': 120,
 'toky

In [ ]:
# prepare a dataframe
words_df = pd.DataFrame({'word':list(words_dict.keys()), 'count':list(words_dict.values())})

# sort words by their count in increasing order
words_df = words_df.sort_values(by = ['count'])

# reset dataframe index
words_df.reset_index(inplace = True, drop=True)

In [ ]:
# words with least frequency
words_df.head()

,word,count
0,sb,1
1,mgll,1
2,922,1
3,979,1
4,norquest,1


In [ ]:
# words with highest frequency
words_df.tail()

,word,count
13854,said,4649
13855,in,5070
13856,to,6337
13857,of,6671
13858,the,12496


In [ ]:
# vocabulary size
len(words_df)

13859

## 2. Creating N-grams of the dialogue

In [ ]:
# creating an empty dataframe
dataset = pd.DataFrame()

# adding cleaned sentences in the dataframe
dataset['Sentences'] = dialogs_clean

# first 20 cleaned sentences
dataset.head(20)

,Sentences
0,asian exporters fear damage from u s japan r...
1,they told reuter correspondents in asian capit...
2,but some exporters said that while the conflic...
3,the u s has said it will impose 300 mln dlrs...
4,unofficial japanese estimates put the impact o...
5,we wouldn ' t be able to do business said a ...
6,if the tariffs remain in place for any length...
7,in taiwan businessmen and officials are also ...
8,we are aware of the seriousness of the u s
9,threat against japan because it serves as a wa...


In [ ]:
# using .split() to get tokens from the sentence
dataset['Sentences'][0].split()

['asian',
 'exporters',
 'fear',
 'damage',
 'from',
 'u',
 's',
 'japan',
 'rift',
 'mounting',
 'trade',
 'friction',
 'between',
 'the',
 'u',
 's',
 'and',
 'japan',
 'has',
 'raised',
 'fears',
 'among',
 'many',
 'of',
 'asia',
 "'",
 's',
 'exporting',
 'nations',
 'that',
 'the',
 'row',
 'could',
 'inflict',
 'far',
 'reaching',
 'economic',
 'damage',
 'businessmen',
 'and',
 'officials',
 'said']

In [ ]:
# function to create unigrams
# taking a sentence as input
def create_unigram(sentence):
    # creating tokens from the sentence
    tokens = sentence.split()
    # empty list to store the unigrams
    unigram_list = []
    # number of unigrams is equal to the number of tokens in the sentence
    for i in range(len(tokens)):
        # appending each unigram in the list
        unigram_list.append(tokens[i:i+1])
    # returning the unigram list for a sentence
    return unigram_list

In [ ]:
create_unigram(dataset['Sentences'][0])

[['asian'],
 ['exporters'],
 ['fear'],
 ['damage'],
 ['from'],
 ['u'],
 ['s'],
 ['japan'],
 ['rift'],
 ['mounting'],
 ['trade'],
 ['friction'],
 ['between'],
 ['the'],
 ['u'],
 ['s'],
 ['and'],
 ['japan'],
 ['has'],
 ['raised'],
 ['fears'],
 ['among'],
 ['many'],
 ['of'],
 ['asia'],
 ["'"],
 ['s'],
 ['exporting'],
 ['nations'],
 ['that'],
 ['the'],
 ['row'],
 ['could'],
 ['inflict'],
 ['far'],
 ['reaching'],
 ['economic'],
 ['damage'],
 ['businessmen'],
 ['and'],
 ['officials'],
 ['said']]

In [ ]:
# function to create bigrams
def create_bigram(sentence):
    tokens = sentence.split()
    bigram_list = []
    # number of bigrams is one less than the number of tokens in the sentence
    for i in range(len(tokens)-1):
        bigram_list.append(tokens[i:i+2])
    return bigram_list

In [ ]:
# function to create trigrams
def create_trigram(sentence):
    tokens = sentence.split()
    trigram_list = []
    # number of trigrams is two less than the number of tokens in the sentence
    for i in range(len(tokens)-2):
        trigram_list.append(tokens[i:i+3])
    return trigram_list

In [ ]:
# creating unigrams for all the sentences in the dataset
final_unigram = []
# for each sentence
for i in range(dataset.shape[0]):
    # using the defined unigram function to create unigrams
    final_unigram.append(create_unigram(dataset['Sentences'][i]))

# adding the unigram in a seperate column in the dataset
dataset['unigram'] = final_unigram

In [ ]:
# creating bigrams for all the sentences in the dataset
final_bigram = []
for i in range(dataset.shape[0]):
    final_bigram.append(create_bigram(dataset['Sentences'][i]))

dataset['bigram'] = final_bigram

In [ ]:
# creating trigrams for all the sentences in the dataset
final_trigram = []
for i in range(dataset.shape[0]):
    final_trigram.append(create_trigram(dataset['Sentences'][i]))

dataset['trigram'] = final_trigram

In [ ]:
# first 20 rows of the dataset
dataset.head(20)

,Sentences,unigram,bigram,trigram
0,asian exporters fear damage from u s japan r...,"[[asian], [exporters], [fear], [damage], [from...","[[asian, exporters], [exporters, fear], [fear,...","[[asian, exporters, fear], [exporters, fear, d..."
1,they told reuter correspondents in asian capit...,"[[they], [told], [reuter], [correspondents], [...","[[they, told], [told, reuter], [reuter, corres...","[[they, told, reuter], [told, reuter, correspo..."
2,but some exporters said that while the conflic...,"[[but], [some], [exporters], [said], [that], [...","[[but, some], [some, exporters], [exporters, s...","[[but, some, exporters], [some, exporters, sai..."
3,the u s has said it will impose 300 mln dlrs...,"[[the], [u], [s], [has], [said], [it], [will],...","[[the, u], [u, s], [s, has], [has, said], [sai...","[[the, u, s], [u, s, has], [s, has, said], [ha..."
4,unofficial japanese estimates put the impact o...,"[[unofficial], [japanese], [estimates], [put],...","[[unofficial, japanese], [japanese, estimates]...","[[unofficial, japanese, estimates], [japanese,..."
5,we wouldn ' t be able to do business said a ...,"[[we], [wouldn], ['], [t], [be], [able], [to],...","[[we, wouldn], [wouldn, '], [', t], [t, be], [...","[[we, wouldn, '], [wouldn, ', t], [', t, be], ..."
6,if the tariffs remain in place for any length...,"[[if], [the], [tariffs], [remain], [in], [plac...","[[if, the], [the, tariffs], [tariffs, remain],...","[[if, the, tariffs], [the, tariffs, remain], [..."
7,in taiwan businessmen and officials are also ...,"[[in], [taiwan], [businessmen], [and], [offici...","[[in, taiwan], [taiwan, businessmen], [busines...","[[in, taiwan, businessmen], [taiwan, businessm..."
8,we are aware of the seriousness of the u s,"[[we], [are], [aware], [of], [the], [seriousne...","[[we, are], [are, aware], [aware, of], [of, th...","[[we, are, aware], [are, aware, of], [aware, o..."
9,threat against japan because it serves as a wa...,"[[threat], [against], [japan], [because], [it]...","[[threat, against], [against, japan], [japan, ...","[[threat, against, japan], [against, japan, be..."


In [ ]:
# sample sentence
dataset['Sentences'][0]

"asian exporters fear damage from u  s  japan rift mounting trade friction between the u  s  and japan has raised fears among many of asia ' s exporting nations that the row could inflict far  reaching economic damage  businessmen and officials said "

In [ ]:
# unigram of the sentence
dataset['unigram'][0]

[['asian'],
 ['exporters'],
 ['fear'],
 ['damage'],
 ['from'],
 ['u'],
 ['s'],
 ['japan'],
 ['rift'],
 ['mounting'],
 ['trade'],
 ['friction'],
 ['between'],
 ['the'],
 ['u'],
 ['s'],
 ['and'],
 ['japan'],
 ['has'],
 ['raised'],
 ['fears'],
 ['among'],
 ['many'],
 ['of'],
 ['asia'],
 ["'"],
 ['s'],
 ['exporting'],
 ['nations'],
 ['that'],
 ['the'],
 ['row'],
 ['could'],
 ['inflict'],
 ['far'],
 ['reaching'],
 ['economic'],
 ['damage'],
 ['businessmen'],
 ['and'],
 ['officials'],
 ['said']]

In [ ]:
# bigram of the sentence
dataset['bigram'][0]

[['asian', 'exporters'],
 ['exporters', 'fear'],
 ['fear', 'damage'],
 ['damage', 'from'],
 ['from', 'u'],
 ['u', 's'],
 ['s', 'japan'],
 ['japan', 'rift'],
 ['rift', 'mounting'],
 ['mounting', 'trade'],
 ['trade', 'friction'],
 ['friction', 'between'],
 ['between', 'the'],
 ['the', 'u'],
 ['u', 's'],
 ['s', 'and'],
 ['and', 'japan'],
 ['japan', 'has'],
 ['has', 'raised'],
 ['raised', 'fears'],
 ['fears', 'among'],
 ['among', 'many'],
 ['many', 'of'],
 ['of', 'asia'],
 ['asia', "'"],
 ["'", 's'],
 ['s', 'exporting'],
 ['exporting', 'nations'],
 ['nations', 'that'],
 ['that', 'the'],
 ['the', 'row'],
 ['row', 'could'],
 ['could', 'inflict'],
 ['inflict', 'far'],
 ['far', 'reaching'],
 ['reaching', 'economic'],
 ['economic', 'damage'],
 ['damage', 'businessmen'],
 ['businessmen', 'and'],
 ['and', 'officials'],
 ['officials', 'said']]

In [ ]:
# trigram of the sentence
dataset['trigram'][0]

[['asian', 'exporters', 'fear'],
 ['exporters', 'fear', 'damage'],
 ['fear', 'damage', 'from'],
 ['damage', 'from', 'u'],
 ['from', 'u', 's'],
 ['u', 's', 'japan'],
 ['s', 'japan', 'rift'],
 ['japan', 'rift', 'mounting'],
 ['rift', 'mounting', 'trade'],
 ['mounting', 'trade', 'friction'],
 ['trade', 'friction', 'between'],
 ['friction', 'between', 'the'],
 ['between', 'the', 'u'],
 ['the', 'u', 's'],
 ['u', 's', 'and'],
 ['s', 'and', 'japan'],
 ['and', 'japan', 'has'],
 ['japan', 'has', 'raised'],
 ['has', 'raised', 'fears'],
 ['raised', 'fears', 'among'],
 ['fears', 'among', 'many'],
 ['among', 'many', 'of'],
 ['many', 'of', 'asia'],
 ['of', 'asia', "'"],
 ['asia', "'", 's'],
 ["'", 's', 'exporting'],
 ['s', 'exporting', 'nations'],
 ['exporting', 'nations', 'that'],
 ['nations', 'that', 'the'],
 ['that', 'the', 'row'],
 ['the', 'row', 'could'],
 ['row', 'could', 'inflict'],
 ['could', 'inflict', 'far'],
 ['inflict', 'far', 'reaching'],
 ['far', 'reaching', 'economic'],
 ['reaching', 

In [ ]:
model = defaultdict(lambda: defaultdict(lambda: 0))

In [ ]:
model

defaultdict(<function __main__.<lambda>()>, {})

## 3. Building the N-gram Language Model

In [ ]:
# for defining the N-gram model
from collections import Counter, defaultdict

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance
for i in range(dataset.shape[0]):
    # for each trigram pair
    for w1, w2, w3 in create_trigram(dataset['Sentences'][i]):
        # count the occurance of word 3, given word 1 and word 2
        model[(w1, w2)][w3] += 1

In [ ]:
# defined model
model

defaultdict(<function __main__.<lambda>()>,
            {('asian',
              'exporters'): defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'fear': 1}),
             ('exporters',
              'fear'): defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'damage': 1}),
             ('fear',
              'damage'): defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'from': 1}),
             ('damage',
              'from'): defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'u': 1, 'local': 1}),
             ('from',
              'u'): defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'s': 5, 'k': 1}),
             ('u',
              's'): defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {'japan': 6,
                          'and': 34,
                      

## 4. Predicting the next word using N-gram Language Model

In [ ]:
# predict the next word
dict(model["u", "s"])

{'japan': 6,
 'and': 34,
 'move': 1,
 'has': 7,
 'said': 4,
 'products': 5,
 'up': 2,
 'pressure': 3,
 'last': 5,
 'exports': 8,
 'japanese': 5,
 'trade': 44,
 'department': 3,
 'dlrs': 36,
 'securities': 2,
 'glass': 1,
 'commercial': 1,
 'officials': 30,
 'farm': 6,
 'would': 5,
 'to': 6,
 'he': 4,
 'agriculture': 72,
 'market': 7,
 'corn': 12,
 'will': 2,
 'feedgrain': 1,
 'which': 2,
 'limit': 1,
 'based': 2,
 'currency': 13,
 'sanctions': 3,
 'official': 3,
 'not': 3,
 'hopes': 2,
 'investors': 1,
 'dollar': 24,
 'unit': 3,
 'where': 6,
 'charges': 1,
 'decision': 1,
 'chip': 2,
 'concern': 1,
 'complaint': 1,
 'banking': 4,
 'treasury': 10,
 'petrochemical': 2,
 'chemical': 3,
 'economy': 12,
 'dlr': 9,
 'textile': 6,
 'investor': 1,
 'defense': 4,
 'government': 11,
 'no': 2,
 'a': 6,
 'parent': 1,
 'acquisition': 1,
 'tariffs': 4,
 'semiconductor': 3,
 'firms': 4,
 'purchase': 1,
 'medical': 2,
 'subsidiary': 6,
 'loan': 1,
 'lawmakers': 2,
 'dairy': 1,
 'interest': 8,
 'engine

In [ ]:
# another example
dict(model["my", "name"])

{}

In [ ]:
# another example
dict(model["how", "are"])

{'european': 1}

In [ ]:
# another example
dict(model["good", "to"])

{'go': 1, 'have': 1, 'very': 1}

### Probabilistic Output

In [ ]:
# creating the unigram list
unigram_dict = {}
for i in tqdm(range(dataset.shape[0])):
    # add word-count pair to the dictionary
    for word in dataset['unigram'][i]:
        # check if the word is already in dictionary
        if word[0] in unigram_dict:
            # increment count of word by 1
            unigram_dict[word[0]] = unigram_dict[word[0]] + 1
        else:
            # add the word to dictionary with count 1
            unigram_dict[word[0]] = 1

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 71925.95it/s]


In [ ]:
# unigram list
unigram_dict

{'asian': 13,
 'exporters': 52,
 'fear': 8,
 'damage': 29,
 'from': 1369,
 'u': 1117,
 's': 2847,
 'japan': 441,
 'rift': 1,
 'mounting': 5,
 'trade': 549,
 'friction': 8,
 'between': 191,
 'the': 12496,
 'and': 4599,
 'has': 974,
 'raised': 70,
 'fears': 13,
 'among': 44,
 'many': 54,
 'of': 6671,
 'asia': 14,
 "'": 2094,
 'exporting': 12,
 'nations': 71,
 'that': 1376,
 'row': 3,
 'could': 291,
 'inflict': 1,
 'far': 55,
 'reaching': 7,
 'economic': 244,
 'businessmen': 15,
 'officials': 190,
 'said': 4649,
 'they': 518,
 'told': 237,
 'reuter': 27,
 'correspondents': 3,
 'in': 5070,
 'capitals': 3,
 'a': 4410,
 'move': 101,
 'against': 270,
 'might': 59,
 'boost': 45,
 'protectionist': 22,
 'sentiment': 10,
 'lead': 96,
 'to': 6337,
 'curbs': 12,
 'on': 1643,
 'american': 126,
 'imports': 242,
 'their': 230,
 'products': 200,
 'but': 650,
 'some': 278,
 'while': 164,
 'conflict': 3,
 'would': 926,
 'hurt': 11,
 'them': 58,
 'long': 119,
 'run': 21,
 'short': 87,
 'term': 120,
 'toky

In [ ]:
# find the overall frequency of words in the corpus
counts = Counter(unigram_dict)
counts

Counter({'asian': 13,
         'exporters': 52,
         'fear': 8,
         'damage': 29,
         'from': 1369,
         'u': 1117,
         's': 2847,
         'japan': 441,
         'rift': 1,
         'mounting': 5,
         'trade': 549,
         'friction': 8,
         'between': 191,
         'the': 12496,
         'and': 4599,
         'has': 974,
         'raised': 70,
         'fears': 13,
         'among': 44,
         'many': 54,
         'of': 6671,
         'asia': 14,
         "'": 2094,
         'exporting': 12,
         'nations': 71,
         'that': 1376,
         'row': 3,
         'could': 291,
         'inflict': 1,
         'far': 55,
         'reaching': 7,
         'economic': 244,
         'businessmen': 15,
         'officials': 190,
         'said': 4649,
         'they': 518,
         'told': 237,
         'reuter': 27,
         'correspondents': 3,
         'in': 5070,
         'capitals': 3,
         'a': 4410,
         'move': 101,
         'against': 2

In [ ]:
# vocabulary size
total_count = len(unigram_dict)
total_count

13859

In [ ]:
# relative frequencies of each word
for word in counts:
    counts[word] /= float(total_count)

counts

Counter({'asian': 0.000938018616061765,
         'exporters': 0.00375207446424706,
         'fear': 0.0005772422252687784,
         'damage': 0.002092503066599322,
         'from': 0.09878057579911971,
         'u': 0.08059744570315319,
         's': 0.2054260769175265,
         'japan': 0.03182047766794141,
         'rift': 7.21552781585973e-05,
         'mounting': 0.0003607763907929865,
         'trade': 0.03961324770906992,
         'friction': 0.0005772422252687784,
         'between': 0.013781658128292084,
         'the': 0.9016523558698318,
         'and': 0.331842124251389,
         'has': 0.07027924092647377,
         'raised': 0.005050869471101811,
         'fears': 0.000938018616061765,
         'among': 0.0031748322389782814,
         'many': 0.0038963850205642543,
         'of': 0.4813478605960026,
         'asia': 0.0010101738942203623,
         "'": 0.15109315246410274,
         'exporting': 0.0008658633379031676,
         'nations': 0.005123024749260408,
         'that'

In [ ]:
# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [ ]:
# predict the next word
my_dict = dict(model["u", "s"])

In [ ]:

# Sorting dictionary using lambda function
sorted_dict = dict(sorted(my_dict.items(), key=lambda x: x[1],reverse =True))

# Printing sorted dictionary
print("Sorted dictionary using lambda is : ", sorted_dict)

Sorted dictionary using lambda is :  {'agriculture': 0.07903402854006586, 'trade': 0.048298572996706916, 'dlrs': 0.03951701427003293, 'and': 0.03732162458836443, 'officials': 0.03293084522502744, 'dollar': 0.026344676180021953, 'currency': 0.014270032930845226, 'corn': 0.013172338090010977, 'economy': 0.013172338090010977, 'government': 0.012074643249176729, 'oil': 0.012074643249176729, 'treasury': 0.010976948408342482, 'dlr': 0.009879253567508232, 'meat': 0.009879253567508232, 'exports': 0.008781558726673985, 'interest': 0.008781558726673985, 'wheat': 0.008781558726673985, 'has': 0.007683863885839737, 'market': 0.007683863885839737, 'house': 0.007683863885839737, 'japan': 0.006586169045005488, 'farm': 0.006586169045005488, 'to': 0.006586169045005488, 'where': 0.006586169045005488, 'textile': 0.006586169045005488, 'a': 0.006586169045005488, 'subsidiary': 0.006586169045005488, 'trust': 0.006586169045005488, 'district': 0.006586169045005488, 'products': 0.005488474204171241, 'last': 0.00

In [ ]:
next(iter(dict.values(model["u", "s"])))

0.006586169045005488

In [ ]:
#for loop for getting first key
for new_s, new_val in sorted(dict(model["u", "s"]).values(),reverse=True):
#print first key
    A=new_s, new_val
#after getting first key break loop
    break

TypeError: cannot unpack non-iterable float object

In [ ]:
print("Sorted dictionary using sorted() and items() is :")
for k, v in sorted_dict:
    print(k, v)

Sorted dictionary using sorted() and items() is :


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
A

('zimmerman', 0.0010976948408342481)

In [ ]:
for key in sorted(steps, key=lambda key: int(key)):

In [ ]:
first_value

0.006586169045005488

In [ ]:
# another example
dict(model["how", "are"])

{'european': 1.0}

In [ ]:
# another example
dict(model["good", "to"])

{'go': 0.3333333333333333,
 'have': 0.3333333333333333,
 'very': 0.3333333333333333}